In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
x = autograd.Variable(torch.Tensor([1., 2., 3]), requires_grad=True)
y = autograd.Variable(torch.Tensor([4., 5., 6]), requires_grad=True)
z = x + y
s = z.sum()

In [11]:
s.backward()

In [14]:
print(x.grad)

Variable containing:
 1
 1
 1
[torch.FloatTensor of size 3]



In [16]:
# nn example
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [18]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

class BoWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
    
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec))

In [19]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

for param in model.parameters():
    print(param)

Parameter containing:

Columns 0 to 9 
-0.0325  0.1950  0.0864  0.1697 -0.1961 -0.1459 -0.0775  0.1957 -0.1386 -0.1035
 0.1483 -0.1061 -0.1854  0.0135  0.0669  0.1624 -0.0324 -0.0168  0.0230 -0.0272

Columns 10 to 19 
-0.1599 -0.0406 -0.1231 -0.0440 -0.0606  0.0666 -0.0405  0.1708  0.0152  0.1358
-0.1411  0.1722 -0.1184  0.1092  0.1180  0.0847  0.1837  0.1188 -0.0732 -0.1597

Columns 20 to 25 
-0.0317 -0.0732  0.0726  0.0096 -0.1159 -0.0222
 0.0754  0.0071  0.1476  0.1432  0.1548  0.1291
[torch.FloatTensor of size 2x26]

Parameter containing:
-0.1628
 0.1293
[torch.FloatTensor of size 2]



In [20]:
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))

label_to_ix = {"SPANISH": 0, "ENGLISH": 1}


In [21]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    for instance, label in data:
        model.zero_grad()
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))
        
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [24]:
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
 0.3315
-0.6325
[torch.FloatTensor of size 2]



In [33]:
## Word embeddings : Encoding Lexical Semantics

In [34]:
torch.manual_seed(1)

In [35]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-2.9718  1.7070 -0.4305 -2.2820  0.5237
[torch.FloatTensor of size 1x5]



In [36]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


In [37]:
trigrams = [
    (
        [test_sentence[i], test_sentence[i+1]],
        test_sentence[i+2]
    ) for i in range(len(test_sentence) - 2)
]

In [39]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [53]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [54]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [59]:
for epoch in range(100):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_function(
            log_probs,
            autograd.Variable(torch.LongTensor([word_to_ix[target]]))
        )
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)

In [61]:
# Exercise: CBOW model

$$\text{-log} p(w_i|C) = \text{-log} Softmax(A(\sum_{w \in C}(q_w)) + b) $$

In [62]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [70]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        out = self.linear(embeds.sum(0))
        log_probs = F.log_softmax(out)
        return log_probs

In [72]:

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


In [81]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [82]:
for epoch in range(100):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_var = make_context_vector(context, word_to_ix)
        
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_function(
            log_probs,
            autograd.Variable(torch.LongTensor([word_to_ix[target]]))
        )
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)

In [83]:
print(losses)

[
 276.0509
[torch.FloatTensor of size 1]
, 
 250.1073
[torch.FloatTensor of size 1]
, 
 228.3769
[torch.FloatTensor of size 1]
, 
 209.8462
[torch.FloatTensor of size 1]
, 
 193.8286
[torch.FloatTensor of size 1]
, 
 179.8552
[torch.FloatTensor of size 1]
, 
 167.6058
[torch.FloatTensor of size 1]
, 
 156.8364
[torch.FloatTensor of size 1]
, 
 147.3302
[torch.FloatTensor of size 1]
, 
 138.8847
[torch.FloatTensor of size 1]
, 
 131.3206
[torch.FloatTensor of size 1]
, 
 124.4900
[torch.FloatTensor of size 1]
, 
 118.2771
[torch.FloatTensor of size 1]
, 
 112.5920
[torch.FloatTensor of size 1]
, 
 107.3639
[torch.FloatTensor of size 1]
, 
 102.5357
[torch.FloatTensor of size 1]
, 
 98.0598
[torch.FloatTensor of size 1]
, 
 93.8957
[torch.FloatTensor of size 1]
, 
 90.0086
[torch.FloatTensor of size 1]
, 
 86.3682
[torch.FloatTensor of size 1]
, 
 82.9480
[torch.FloatTensor of size 1]
, 
 79.7254
[torch.FloatTensor of size 1]
, 
 76.6805
[torch.FloatTensor of size 1]
, 
 73.7959
[torch.

In [76]:
## LSTMs

In [84]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [88]:
lstm = nn.LSTM(3, 3)
inputs = [autograd.Variable(torch.randn((1, 3))) for _ in range(5)]
hidden = (autograd.Variable(torch.randn(1, 1, 3)), 
          autograd.Variable(torch.randn(1, 1, 3)))

for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(hidden)

(Variable containing:
(0 ,.,.) = 
 -0.3950  0.0929  0.0992
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.7724  0.2366  0.1357
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
1.00000e-02 *
  -8.1686  6.2331  5.4420
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.3853  0.2172  0.2259
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
 -0.1209  0.1152 -0.0173
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.3142  0.5087 -0.0371
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
 -0.1254  0.1470 -0.2288
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.2096  0.6587 -0.3916
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
 -0.0935  0.1743  0.1439
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.2359  0.4571  0.3482
[torch.FloatTensor of size 1x1x3]
)


In [87]:
hidden

(Variable containing:
 (0 ,.,.) = 
   0.0202  0.0141  0.3499
 [torch.FloatTensor of size 1x1x3], Variable containing:
 (0 ,.,.) = 
   0.0395  0.0313  0.7842
 [torch.FloatTensor of size 1x1x3])